### Just a tweak to use sublime shortcuts

In [1]:
%%javascript
require(["codemirror/keymap/sublime", "notebook/js/cell", "base/js/namespace"],
    function(sublime_keymap, cell, IPython) {
        cell.Cell.options_default.cm_config.keyMap = 'sublime';
        var cells = IPython.notebook.get_cells();
        for(var cl=0; cl< cells.length ; cl++){
            cells[cl].code_mirror.setOption('keyMap', 'sublime');
        }
    }
);

<IPython.core.display.Javascript object>

In [2]:
# Install tensorflow provided datasets - uncomment on the first run
# !pip install -U tensorflow_datasets

In [5]:
# Necessary imports
import tensorflow as tf

# Import TensorFlow Datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

# Logging
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

### Fashion MNIST dataset

The Fashion MNIST dataset comes as a dictionary, with an already performed train and test dataset split. Metadata provide information about the dataset.

In [24]:
# Load dataset and metadata
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [25]:
# Extract label from metadata
class_names = metadata.features['label'].names
print("Class names: {}".format(class_names))

Class names: ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [26]:
# Read information on train and test dataset
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

Number of training examples: 60000
Number of test examples:     10000


### Preprocessing

In [27]:
from local import preprocessing as pr

In [28]:
# Plot of a single image
# Take a single image, and remove the color dimension by reshaping
for image, label in test_dataset.take(1):
    break
print('Image shape before color dimension removal:', np.shape(image))
image = image.numpy().reshape((28,28))
print('Image shape after color dimension removal: ', np.shape(image))

Image shape before color dimension removal: (28, 28, 1)
Image shape after color dimension removal:  (28, 28)


In [29]:
# The map function applies the normalize function to each element in the train and test datasets
# The normalize function converts the integer value in [0,255] to a float value in [0,1]
train_dataset =  train_dataset.map(pr.normalize)
test_dataset  =  test_dataset.map(pr.normalize)

In [30]:
# The first time you use the dataset, the images will be loaded from disk
# Caching will keep them in memory, making training faster
train_dataset =  train_dataset.cache()
test_dataset  =  test_dataset.cache()

In [31]:
# Model definition
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [32]:
# Compiling the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [33]:
# Training the model
BATCH_SIZE = 32
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

In [34]:
train_history = model.fit(train_dataset, epochs=10, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE)) 

Epoch 1/10
1875/1875 [==============================] - 89s 46ms/step - loss: 0.5570 - accuracy: 0.7952
Epoch 2/10
1875/1875 [==============================] - 94s 50ms/step - loss: 0.2718 - accuracy: 0.9014
Epoch 3/10
1875/1875 [==============================] - 86s 46ms/step - loss: 0.2187 - accuracy: 0.9198
Epoch 4/10
1875/1875 [==============================] - 84s 45ms/step - loss: 0.1912 - accuracy: 0.9290
Epoch 5/10
1875/1875 [==============================] - 83s 45ms/step - loss: 0.1566 - accuracy: 0.9423
Epoch 6/10
1875/1875 [==============================] - 83s 44ms/step - loss: 0.1286 - accuracy: 0.9518
Epoch 7/10
1875/1875 [==============================] - 83s 44ms/step - loss: 0.1169 - accuracy: 0.9566
Epoch 8/10
1875/1875 [==============================] - 83s 44ms/step - loss: 0.0966 - accuracy: 0.9641
Epoch 9/10
1875/1875 [==============================] - 83s 44ms/step - loss: 0.0808 - accuracy: 0.9690
Epoch 10/10
1875/1875 [==============================] - 83s 44m